<a href="https://colab.research.google.com/github/microprediction/winningnotebooks/blob/main/LLM_Quinellas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install winning
!pip install pandas
!pip install scipy

# Luce's Choice Axiom versus the Standard Normal Race model
The methodology is as follows.

1.   Ask an LLM to assign probabilities $p_i$ to a set A of tokens
2.   Ask an LLM to assign probabilities to a subset $B \subset A$ of tokens

We then try to predict the subset probabilities in two ways:

1.   A simple renormalization (Luce Choice Axiom):  $p_i/(\sum_{j\in B} p_j)$
2.   The Standard Normal Race model: Set $X_i \sim N(a_i,1)$ where $a_i$ are calibrated to the $p_i$ using the ability transform.  

We then compare the errors.




## A contest model for choice

Luce is trivial. Let's just implement the second here using the `winning` package:;

In [2]:
from winning.std_calibration import std_state_price_implied_ability, STD_UNIT, STD_L, STD_SCALE, std_ability_implied_state_prices
def ability_implied_subrace_probabilities(race:dict, runners:[str])-> dict:
     #   Subrace probabilities
     probs = list(race.values())
     names = list(race.keys())
     abilities = std_state_price_implied_ability(probs, unit=STD_UNIT, L=STD_L, scale=STD_SCALE)
     sub_names = [nm for nm in names if nm in runners]
     sub_abil = [a for nm, a in zip(names,abilities) if nm in runners]
     sub_prob = implied_probabilities = std_ability_implied_state_prices(ability=sub_abil,unit=STD_UNIT, L=STD_L, scale=STD_SCALE)
     implied = dict( zip(sub_names,sub_prob) )
     return implied


race = {'red':0.5,'green':0.3,'blue':0.2}

runners = ['green','red']
implied = ability_implied_subrace_probabilities(race,runners )
implied

{'red': 0.6169905666139499, 'green': 0.38396120015303187}

# Quinella pricing

In [91]:
import numpy as np
from winning.lattice_conventions import STD_L, STD_A
from winning.lattice import skew_normal_density, densities_from_offsets, get_the_rest, _loser_of_two_pdf,\
    beats, winner_of_many, cdf_to_pdf
from winning.lattice_calibration import state_price_implied_ability


def compute_skew_normal_quinellas(p:[float], L=551, a=0):
    """ Produce quinella table, and also return densities

    :param p:  Vector of state prices
    :param L:  500 by default, half that is probably fine
    :return: quinellas, densities
    """

    # Calibration
    unit = 1.0
    density = skew_normal_density(L=L, unit=unit, loc=0, scale=50.0, a=a)
    offsets = state_price_implied_ability(prices=p, density=density)
    densities = densities_from_offsets(density, offsets)
    densityAll, multiplicityAll = winner_of_many(densities)

    n = len(p)
    quinellas = np.ndarray(shape=(n, n))
    for h0 in range(n):
        density0 = densities[h0]
        cdfRest0, multiplicityRest0 = get_the_rest(density=density0, densityAll=densityAll,
                                                   multiplicityAll=multiplicityAll, cdf=None, cdfAll=None)
        for h1 in range(n):
            if h1 > h0:
                density1 = densities[h1]
                cdfRest01, multiplicityRest01 = get_the_rest(density=density1, densityAll=None,
                                                             multiplicityAll=multiplicityRest0, cdf=None,
                                                             cdfAll=cdfRest0)
                pdfRest01 = cdf_to_pdf(cdfRest01)
                loser01, loser_multiplicity01 = _loser_of_two_pdf(density0, density1)
                quinellas[h0, h1] = 1 / beats(loser01, loser_multiplicity01, pdfRest01, multiplicityRest01)
                quinellas[h1, h0] = quinellas[h0, h1]
    return quinellas, densities

q,_ = compute_skew_normal_quinellas(p=[0.5,0.3,0.1,0.1,0.1,0.001,0.001,0.001,0.001,0.001,0.001])
q[:4,:4]

array([[ 1.        ,  3.01759035,  8.21247528,  8.21247528],
       [ 3.01759035,  1.        , 14.55937804, 14.55937804],
       [ 8.21247528, 14.55937804,  1.        , 42.33440703],
       [ 8.21247528, 14.55937804, 42.33440703,  1.        ]])

In [93]:
def compute_harville_quinellas(p):
    """
    Compute Harville Quinellas (joint probabilities for unordered pairs) from individual probabilities assuming independence.

    Args:
        p (list of float): List of individual probabilities for each state/event. Should sum to <=1.

    Returns:
        dict: Dictionary where keys are frozensets representing unordered pairs of states,
              and values are the joint probabilities P({i, j}).
    """
    from itertools import combinations

    n = len(p)
    quinellas = {}

    # Compute unnormalized joint probabilities
    for i, j in combinations(range(n), 2):
        joint_prob = p[i] * p[j]
        quinellas[frozenset([i, j])] = joint_prob

    # Normalize the joint probabilities so that their sum equals the total probability of any two events occurring
    total_joint_prob = sum(quinellas.values())
    if total_joint_prob > 0:
        quinellas = {pair: prob / total_joint_prob for pair, prob in quinellas.items()}

    return quinellas

compute_harville_quinellas(p=[0.5,0.3,0.2])

{frozenset({0, 1}): 0.48387096774193544,
 frozenset({0, 2}): 0.32258064516129037,
 frozenset({1, 2}): 0.1935483870967742}

In [ ]:
!pip install transformers numpy pandas scipy

# Single state selection probabilities

In [95]:
import pandas as pd
import numpy as np
import itertools

# Define an expanded list of single-word Western U.S. states
single_word_states = [
    'california', 'oregon', 'washington', 'arizona', 'nevada',
    'utah', 'idaho', 'montana', 'wyoming', 'colorado',
    'new mexico', 'texas', 'oklahoma', 'kansas', 'south dakota',
    'north dakota', 'nebraska', 'iowa', 'missouri', 'arkansas'
]

# Filter to include only single-word states to avoid tokenizer issues
single_word_states = [state for state in single_word_states if ' ' not in state]

# Define the sentence template with one state fixed and one [MASK]
fixed_state = 'california'
sentence_template = f"I visited the state of {fixed_state.capitalize()} and [MASK] last year, and they are my two favorite states in the United States."

# Step 1: Get Individual Probabilities (P(i))
probs_individual = fill_in_missing_word(sentence=sentence_template, exclude_words=None, top_k=200)

# Ensure all state names are in lowercase to match the list
probs_individual = {k.lower(): v for k, v in probs_individual.items()}

# Step 2: Filter probabilities to only include our single-word states
probs_individual_filtered = {state: probs_individual[state] for state in single_word_states if state in probs_individual}

# Check if any states are missing
missing_states = set(single_word_states) - set(probs_individual_filtered.keys())
if missing_states:
    print(f"Warning: The following states were not predicted by BERT and will be excluded: {missing_states}")

# Step 3: Normalize the probabilities to sum to 1
total_prob_individual = sum(probs_individual_filtered.values())
if total_prob_individual == 0:
    raise ValueError("Total individual probability is zero. Check the BERT predictions.")
probs_individual_normalized = {state: prob / total_prob_individual for state, prob in probs_individual_filtered.items()}

# Display top states by probability
df_individual = pd.DataFrame(
    sorted(probs_individual_normalized.items(), key=lambda x: x[1], reverse=True),
    columns=['State', 'Probability']
)
print("Individual Probabilities (P(i)):")
print(df_individual)
print("\n")


Individual Probabilities (P(i)):
         State  Probability
0      arizona     0.413279
1       nevada     0.189528
2       oregon     0.173016
3        texas     0.060147
4     colorado     0.034071
5        idaho     0.030942
6      wyoming     0.020346
7         utah     0.016122
8     arkansas     0.013266
9      montana     0.012060
10  california     0.009661
11      kansas     0.007988
12  washington     0.005856
13    nebraska     0.005247
14        iowa     0.005216
15    oklahoma     0.002860
16    missouri     0.000395




# Conditiona state probabilities

In [96]:
import pandas as pd
import numpy as np
import itertools

# Assuming the initial setup code is already executed:
# - single_word_states defined and filtered
# - fixed_state defined as 'california'
# - sentence_template defined
# - probs_individual computed
# - probs_individual_filtered computed
# - probs_individual_normalized computed and displayed

# Step 4: Compute Conditional Probabilities (P(j|i))
conditional_probabilities = {}
for state_i in single_word_states:
    if state_i not in probs_individual_normalized:
        continue  # Skip states that were not predicted

    # Corrected Sentence: Fix state_i and mask only the second state
    sentence = f"I visited the state of {state_i.capitalize()} and [MASK] last year, and they are my two favourite states in the United States."
    print(f"Constructed Sentence for P(j|{state_i}):")
    print(sentence)

    # Get predictions for [MASK]
    probs_j_given_i = fill_in_missing_word(sentence=sentence, exclude_words=[state_i], top_k=200)

    # Ensure all state names are in lowercase to match the list
    probs_j_given_i = {k.lower(): v for k, v in probs_j_given_i.items()}

    # Filter to include only our single-word states and exclude state_i
    probs_j_filtered = {state_j: prob for state_j, prob in probs_j_given_i.items()
                        if state_j in single_word_states and state_j != state_i}

    # Debug: Print conditional probabilities for each state_i
    print(f"Conditional Probabilities P(j|{state_i}):")
    if not probs_j_filtered:
        print(f"No valid predictions for P(j|{state_i}). Assigning zero probabilities.\n")
        # Assign zero probabilities for all possible j
        conditional_probabilities[state_i] = {state_j: 0 for state_j in single_word_states if state_j != state_i}
        continue

    # Normalize the conditional probabilities
    total_prob_j = sum(probs_j_filtered.values())
    if total_prob_j == 0:
        print(f"Warning: No valid predictions for P(j|{state_i}). Assigning zero probabilities.\n")
        probs_j_normalized = {state_j: 0 for state_j in probs_j_filtered}
    else:
        probs_j_normalized = {state_j: prob / total_prob_j for state_j, prob in probs_j_filtered.items()}

    # Debug: Print normalized conditional probabilities
    df_conditional = pd.DataFrame(
        sorted(probs_j_normalized.items(), key=lambda x: x[1], reverse=True),
        columns=['State_j', 'P(j|i)']
    )
    print(df_conditional)
    print("\n")

    conditional_probabilities[state_i] = probs_j_normalized

# Step 5: Compute Exacta Probabilities (P(i, j) = P(i) * P(j|i))
exacta_probabilities = {}
for state_i, probs_j in conditional_probabilities.items():
    for state_j, p_j_given_i in probs_j.items():
        exacta_probabilities[(state_i, state_j)] = probs_individual_normalized[state_i] * p_j_given_i

# Step 6: Compute Quinella Probabilities (P({i, j}) = P(i, j) + P(j, i))
quinella_probabilities = {}
for (state_i, state_j), p_ij in exacta_probabilities.items():
    # To ensure unordered pairs, use frozenset
    pair = frozenset([state_i, state_j])
    if pair in quinella_probabilities:
        quinella_probabilities[pair] += p_ij
    else:
        quinella_probabilities[pair] = p_ij

# Normalize quinella probabilities
total_quinella_prob = sum(quinella_probabilities.values())
if total_quinella_prob == 0:
    raise ValueError("Total quinella probability is zero. Check the exacta probabilities.")
quinella_probabilities_normalized = {pair: prob / total_quinella_prob for pair, prob in quinella_probabilities.items()}



Constructed Sentence for P(j|california):
I visited the state of California and [MASK] last year, and they are my two favourite states in the United States.
Conditional Probabilities P(j|california):
       State_j    P(j|i)
0      arizona  0.419381
1       oregon  0.204331
2       nevada  0.165363
3        texas  0.060958
4     colorado  0.033385
5        idaho  0.026155
6      wyoming  0.017378
7     arkansas  0.014524
8      montana  0.013408
9         utah  0.012697
10  washington  0.009340
11      kansas  0.008910
12        iowa  0.005633
13    nebraska  0.004618
14    oklahoma  0.003455
15    missouri  0.000464


Constructed Sentence for P(j|oregon):
I visited the state of Oregon and [MASK] last year, and they are my two favourite states in the United States.
Conditional Probabilities P(j|oregon):
       State_j    P(j|i)
0   california  0.347736
1   washington  0.317100
2        idaho  0.132629
3       nevada  0.063299
4      montana  0.037887
5     colorado  0.024804
6         

In [97]:
# Step 7: Compute Theoretical Joint Probabilities Using Harville's Method
# Assuming compute_harville_quinellas takes a list of individual probabilities and returns a dict of frozenset pairs
p_list = [probs_individual_normalized[state] for state in single_word_states if state in probs_individual_normalized]
harville_quinellas = compute_harville_quinellas(p=p_list)

In [104]:
# Step 8: Compute Theoretical Joint Probabilities Using Skew-Normal Distribution
# Assuming skew_normal_quinellas takes a list of individual probabilities and returns a matrix
q_matrix, _ = compute_skew_normal_quinellas(p=p_list, L=251, a=0)
q_matrix[:4,:4]

df_skew = pd.DataFrame([
      {'Pair': ', '.join(sorted(pair)), 'Probability_Skew_Normal': prob} for pair, prob in skew_normal_quinellas_named.items()
  ])
print("Skew-Normal Quinellas (First 4 Pairs):")
print(df_skew.head(4))
print("\n")

# Normalize skew_normal_quinellas if necessary
total_skew = sum(skew_normal_quinellas.values())
if total_skew == 0:
    print("Total skew-normal quinella probability is zero.")
    skew_normal_quinellas_normalized = skew_normal_quinellas_named
else:
    skew_normal_quinellas_normalized = {pair: prob / total_skew for pair, prob in skew_normal_quinellas_named.items()}

# Update df_skew with normalized probabilities
df_skew['Probability_Skew_Normal'] = df_skew['Probability_Skew_Normal'].apply(lambda x: x / total_skew if total_skew > 0 else 0)


NameError: name 'skew_normal_quinellas_named' is not defined

In [99]:
# Convert skew_normal_quinellas matrix to quinella_probabilities_skew dict
skew_normal_quinellas = {}
n = len(p_list)
for i in range(n):
    for j in range(n):
        if i != j:
            pair = frozenset([single_word_states[i], single_word_states[j]])
            try:
                skew_normal_quinellas[pair] = 1/q_matrix[i, j]
            except IndexError:
                print(f"IndexError for pair {pair}: q_matrix[{i}, {j}] is out of bounds.")
                skew_normal_quinellas[pair] = 0  # Assign zero or handle appropriately

# Normalize skew_normal_quinellas
total_skew = sum(skew_normal_quinellas.values())
if total_skew == 0:
    print("Total skew-normal quinella probability is zero.")
    skew_normal_quinellas_normalized = skew_normal_quinellas
else:
    skew_normal_quinellas_normalized = {pair: prob / total_skew for pair, prob in skew_normal_quinellas.items()}

skew_normal_quinellas

{frozenset({'california', 'oregon'}): 0.005111879165611478,
 frozenset({'california', 'washington'}): 0.0002453526736954661,
 frozenset({'arizona', 'california'}): 0.012871234021167468,
 frozenset({'california', 'nevada'}): 0.005590418854005174,
 frozenset({'california', 'utah'}): 0.0005981799373887171,
 frozenset({'california', 'idaho'}): 0.0010622255374262252,
 frozenset({'california', 'montana'}): 0.00046341795692141986,
 frozenset({'california', 'wyoming'}): 0.000734090579772625,
 frozenset({'california', 'colorado'}): 0.0011566136604260198,
 frozenset({'california', 'texas'}): 0.0019178390947369147,
 frozenset({'california', 'oklahoma'}): 0.0001308859625168634,
 frozenset({'california', 'kansas'}): 0.00032247055220238516,
 frozenset({'california', 'nebraska'}): 0.00022275021508048992,
 frozenset({'california', 'iowa'}): 0.0002216060361925625,
 frozenset({'california', 'missouri'}): 3.4644050245269346e-05,
 frozenset({'arkansas', 'california'}): 0.0005039169880002704,
 frozenset({'

In [101]:
df_empirical = pd.DataFrame([
    {'Pair': ', '.join(pair), 'Probability_Empirical': prob} for pair, prob in quinella_probabilities_normalized.items()
])
df_empirical.head(4)

,Pair,Probability_Empirical
0,"arizona, california",0.182343
1,"california, oregon",0.062138
2,"california, nevada",0.056692
3,"texas, california",0.008869


In [102]:
# Merge DataFrames on 'Pair'
df_comparison = df_empirical.merge(df_harville, on='Pair', how='outer')
df_comparison = df_comparison.merge(df_skew, on='Pair', how='outer')

# Fill NaN with 0
df_comparison.fillna(0, inplace=True)

# Calculate Differences
df_comparison['Difference_Harville'] = df_comparison['Probability_Empirical'] - df_comparison['Probability_Harville']
df_comparison['Abs_Difference_Harville'] = df_comparison['Difference_Harville'].abs()

df_comparison['Difference_Skew_Normal'] = df_comparison['Probability_Empirical'] - df_comparison['Probability_Skew_Normal']
df_comparison['Abs_Difference_Skew_Normal'] = df_comparison['Difference_Skew_Normal'].abs()

# Calculate RMSE for both models
rmse_harville = np.sqrt(np.mean(df_comparison['Difference_Harville'] ** 2))
rmse_skew_normal = np.sqrt(np.mean(df_comparison['Difference_Skew_Normal'] ** 2))

print(f"Root Mean Square Error (RMSE) - Harville Quinellas: {rmse_harville:.6f}")
print(f"Root Mean Square Error (RMSE) - Skew Normal Quinellas: {rmse_skew_normal:.6f}\n")

# Display the top 10 pairs with the largest absolute differences for Harville
print("Top 10 Pairs with Largest Absolute Differences (Harville):")
print(df_comparison.sort_values('Abs_Difference_Harville', ascending=False).head(10))

# Display the top 10 pairs with the largest absolute differences for Skew Normal Quinella
print("\nTop 10 Pairs with Largest Absolute Differences (Skew Normal Quinella):")
print(df_comparison.sort_values('Abs_Difference_Skew_Normal', ascending=False).head(10))


NameError: name 'df_skew' is not defined

## Experimental Setup...